In [19]:
#포트 열린채로 예외 떴을 때 전용
ser.close()
print("Serial port closed.")

Serial port closed.


In [21]:
import serial
import time
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from rnn_model import RNNModel
from custom_dataset import CustomDataset
from padding import PaddingCollate

# 저장된 모델 파일 경로
model_file = 'rnn_model.pth'

# 모델 불러오기
model = RNNModel(input_size=2, hidden_size=50, output_size=1, num_layers=2)
model.load_state_dict(torch.load(model_file))
model.eval()  # 평가 모드로 설정

# 시리얼 통신 설정
ser = serial.Serial('COM8', 9600)
time.sleep(2)

# 발음 위치 예측 함수 정의
def predict_pronunciation(input_pin1, input_pin2):
    with torch.no_grad():  # 그라디언트 계산 비활성화
        # 최대 길이에 맞춰서 패딩 추가
        max_length = max(len(input_pin1), len(input_pin2))
        input_pin1_padded = nn.functional.pad(input_pin1, (0, max_length - len(input_pin1)))
        input_pin2_padded = nn.functional.pad(input_pin2, (0, max_length - len(input_pin2)))
        # 모델 입력 준비
        inputs = torch.stack([input_pin1_padded, input_pin2_padded], dim=1).float()  # 차원 변경
        inputs = inputs.unsqueeze(0)  # 배치 차원 추가
        # 모델 예측
        output = model(inputs)
        # 예측 결과를 라벨로 변환
        predicted_label = convert_to_label(output.item())
        return predicted_label

# 모델 출력을 라벨로 변환하는 함수 정의
def convert_to_label(prediction):
    # 각 발음 위치의 범위 정의
    label_ranges = {
        '양순음(bilabial)': (0, 1),
        '치조음(alveolar)': (1, 2),
        '연구개음(soft_palate)': (2, 3),
        '경구개음(palate)': (3, 4),
        '후두음(guttural)': (4, 5)
    }
    # 모델 출력 값을 라벨로 변환
    for label, (start, end) in label_ranges.items():
        if start <= prediction < end:
            return label
    return "Unknown"  # 알 수 없는 경우

try:
    while True:
        if ser.inWaiting() > 0:
            line = ser.readline().decode('utf-8').rstrip()  # 데이터 읽기
            if "Arduino" in line:
                print(line)
                
            if "Stop" in line:
                print(line)
                ser.close()
                break
                
            if "Start measurement" in line:
                print(line)
                current_measurement = {"pin1": [], "pin2": []}  # 새로운 측정을 위해 초기화
            elif "End measurement" in line:
                print(line)
                # 발음 위치 예측
                predicted_position = predict_pronunciation(torch.tensor(current_measurement['pin1']), 
                                                            torch.tensor(current_measurement['pin2']))
                print("예상되는 조음 발음 위치:", predicted_position)
            elif line.startswith("pin1: "):
                value = int(line.split(": ")[1])
                current_measurement["pin1"].append(value)
            elif line.startswith("pin2: "):
                value = int(line.split(": ")[1])
                current_measurement["pin2"].append(value)
            
except KeyboardInterrupt:
    ser.close()
    print("Serial port closed.")


--------Arduino ON--------
Start measurement
End measurement. (1.71sec)
예상되는 조음 발음 위치: 치조음(alveolar)
Start measurement
End measurement. (1.91sec)
예상되는 조음 발음 위치: 치조음(alveolar)
Start measurement
End measurement. (1.10sec)
예상되는 조음 발음 위치: 치조음(alveolar)
Start measurement
End measurement. (1.30sec)
예상되는 조음 발음 위치: 연구개음(soft_palate)
Stop
